In [121]:
import time
import datetime
import os
import paramiko   #sftp파일전송목적
paramiko.util.log_to_file('/home/tangosvc/code/data/paramiko.log')   #paramiko패키지에 대한 log파일 생성

In [133]:
class TEOS_filegetput:
        def __init__(self, ML_dirnm, TEOS_indirnm, TEOS_outdirnm, time):
            # ML서버내 파일관리폴더(Request, Result모두 동일폴더에서 관리함)
            self.ML_dirnm=ML_dirnm
            # TEOS관리폴더(Request, Result 별도로 관리함)
            self.TEOS_indirnm=TEOS_indirnm
            self.TEOS_outdirnm=TEOS_outdirnm
            # 시간기준으로 폴더생성/파일관리
            self.time=time

        # dirnm에 있는 filenm으로 시작하는 모든 파일목록 생성
        def get_filelist(self, dirnm, filenm):
            file_list=[f for f in os.listdir(dirnm) if f.startswith(filenm)]
            return(file_list)
        
        # 파일존재시 파일명리턴 부재시 공백리턴
        def get_fileexist(self, dirnm, filenm):
            if os.path.exists(dirnm+'/'+filenm):
                return(filenm)
            else:
                return('')    
        
        # TEOS서버접속
        def conn_TEOS_svr(self):
            host='90.90.220.8'
            port=10003
            transport=paramiko.Transport((host,port))
            password='@Son1234!1'
            username='son'
            transport.connect(username=username, password=password)
            sftp=paramiko.SFTPClient.from_transport(transport)
            return(sftp, transport)

        # put명령실행            
        def do_requestfile_put(self, sftp, transport, putfilenm):
            fromfile=self.ML_dirnm+'/'+self.time+'/'+putfilenm
            tofile=self.TEOS_indirnm+'/'+self.time+'/'+putfilenm
            
            # TEOS서버에 복사할 폴더가 존재하지 않으면 생성함
            dirlist=sftp.listdir(self.TEOS_indirnm)             
            if not self.time in dirlist:
                sftp.mkdir(self.TEOS_indirnm+'/'+self.time)

            # ML서버에 복사할 파일이 존재하지 않으면 실행종료
            temp=os.path.split(fromfile)
            if self.get_fileexist(temp[0], temp[1])!='':
                sftp.put(fromfile, tofile)
            else:
                print('ML서버에 Put대상 파일이 없어 TEOS서버에 Put명령을 하지 못했습니다.')                

        # get명령실행    
        def do_resultfile_get(self, sftp, transport, getfilenm):         
            fromfile=self.TEOS_outdirnm+'/'+self.time+'/'+getfilenm
            tofile=self.ML_dirnm+'/'+self.time+'/'+getfilenm
            
            # ML서버에 복사해올 폴더가 존재하지 않으면 생성함
            if not os.path.exists(self.ML_dirnm+'/'+self.time):
                os.mkdir(self.ML_dirnm+'/'+self.time)
                print(self.ML_dirnm+'/'+self.time)

            # self.time이 TEOS결과출력 폴더에 존재하는 경우에만 get명령실행
            if self.time in sftp.listdir(self.TEOS_outdirnm):
                # TEOS서버에 복사대상 파일이 존재하지 않으면 3번 시행함
                dirlist=sftp.listdir(self.TEOS_outdirnm+'/'+self.time)
                print(dirlist)
                for i in range(3):
                    if getfilenm in dirlist:      
                        sftp.get(fromfile, tofile)
                    else:
                        print(getfilenm+'이 없어 30초 대기후 다시 확인합니다.')
                        time.sleep(30)
            else:
                print(self.TEOS_outdirnm+'/'+self.time+'폴더가 존재하지 않아 TEOS결과파일을 get할 수 없습니다.')
            
        def close_TEOS_svr(self, sftp, transport):
            sftp.close()
            transport.close()

In [130]:
def RequestFile_Put(filenm): 
    ML_dirnm='/home/tangosvc/code/data'
    TEOS_indirnm='/disk2/interworking_data/SON/EngAnalysis/request'        
    TEOS_outdirnm='/disk2/interworking_data/SON/EngAnalysis/result'
    # 임의로 설정한 시간값
#     time='20191017'
    time=datetime.datetime.now().strftime('%Y%m%d')    

    try:
        a=TEOS_filegetput(ML_dirnm, TEOS_indirnm, TEOS_outdirnm, time)
        sftp, transport=a.conn_TEOS_svr()      
        a.do_requestfile_put(sftp, transport, filenm)
        a.close_TEOS_svr(sftp, transport)
        return(1)
        
    except:
        print('Error : TEOS server connection')
        a.close_TEOS_svr(sftp, transport)  
        return(0)

ML서버에 Put대상 파일이 없어 TEOS서버에 Put명령을 하지 못했습니다.


In [134]:
def ResultFile_Get(filenm): 
    ML_dirnm='/home/tangosvc/code/data'
    TEOS_indirnm='/disk2/interworking_data/SON/EngAnalysis/request'        
    TEOS_outdirnm='/disk2/interworking_data/SON/EngAnalysis/result'
    # 임의로 설정한 시간값
    #time='20191018'
    time=datetime.datetime.now().strftime('%Y%m%d')    

    try:
        a=TEOS_filegetput(ML_dirnm, TEOS_indirnm, TEOS_outdirnm, time)
        sftp, transport=a.conn_TEOS_svr()
        a.do_resultfile_get(sftp, transport, filenm)
        a.close_TEOS_svr(sftp, transport)
        return(1)

    except:
        print('Error : TEOS server connection')
        a.close_TEOS_svr(sftp, transport)
        return(0)

/disk2/interworking_data/SON/EngAnalysis/result/20191021폴더가 존재하지 않아 TEOS결과파일을 get할 수 없습니다.


In [143]:
if __name__=='__main__':
    Sim_Stop_Time_From='00:05'
    Sim_Stop_Time_To='23:25'
    Cur_Time=datetime.datetime.now().strftime('%H:%M') 
    
    while (Cur_Time>Sim_Stop_Time_From) & (Cur_Time<Sim_Stop_Time_To):
        print(Cur_Time)
        filenm='SON_TO_TEOS_RL_REQUEST_20191017_2.dat'
        Putresult=RequestFile_Put(filenm)  

        #Put명령이 성공했을 경우
        if Putresult==1:
            time.sleep(30)
            filenm='TEOS_TO_SON_RL_RESULT_20191018_[12].dat'
            ResultFile_Get(filenm)
            
        Cur_Time=datetime.datetime.now().strftime('%H:%M')             

16:00
ML서버에 Put대상 파일이 없어 TEOS서버에 Put명령을 하지 못했습니다.
16:00
ML서버에 Put대상 파일이 없어 TEOS서버에 Put명령을 하지 못했습니다.
16:00
ML서버에 Put대상 파일이 없어 TEOS서버에 Put명령을 하지 못했습니다.
16:00
ML서버에 Put대상 파일이 없어 TEOS서버에 Put명령을 하지 못했습니다.
16:00
ML서버에 Put대상 파일이 없어 TEOS서버에 Put명령을 하지 못했습니다.
16:00
ML서버에 Put대상 파일이 없어 TEOS서버에 Put명령을 하지 못했습니다.
16:00
ML서버에 Put대상 파일이 없어 TEOS서버에 Put명령을 하지 못했습니다.
16:00
ML서버에 Put대상 파일이 없어 TEOS서버에 Put명령을 하지 못했습니다.
16:00
ML서버에 Put대상 파일이 없어 TEOS서버에 Put명령을 하지 못했습니다.
16:00
ML서버에 Put대상 파일이 없어 TEOS서버에 Put명령을 하지 못했습니다.
16:00
ML서버에 Put대상 파일이 없어 TEOS서버에 Put명령을 하지 못했습니다.
16:00
ML서버에 Put대상 파일이 없어 TEOS서버에 Put명령을 하지 못했습니다.
16:00
ML서버에 Put대상 파일이 없어 TEOS서버에 Put명령을 하지 못했습니다.
16:00
ML서버에 Put대상 파일이 없어 TEOS서버에 Put명령을 하지 못했습니다.
16:00
ML서버에 Put대상 파일이 없어 TEOS서버에 Put명령을 하지 못했습니다.
16:00
ML서버에 Put대상 파일이 없어 TEOS서버에 Put명령을 하지 못했습니다.
16:00
ML서버에 Put대상 파일이 없어 TEOS서버에 Put명령을 하지 못했습니다.
16:00
ML서버에 Put대상 파일이 없어 TEOS서버에 Put명령을 하지 못했습니다.
16:00
ML서버에 Put대상 파일이 없어 TEOS서버에 Put명령을 하지 못했습니다.
16:00
ML서버에 Put대상 파일이 없어 TEOS서버에 Put명령을 하지 못했습니다.


UnboundLocalError: local variable 'sftp' referenced before assignment

In [140]:
a=datetime.datetime.now().strftime('%H:%M') 
b='15:54'
print(a<b)

False
